In [20]:
#Este script convierte un archivo PST a CSV
import pypff
import csv

def listar_correos(folder, csv_writer):
    count = 0  # Inicializar contador de correos encontrados en esta carpeta
    for item in folder.sub_items:
        # Verificar si es un correo electrónico
        if isinstance(item, pypff.message):
            # Obtener los datos del correo
            asunto = item.subject
            fecha = item.delivery_time
            enviado_por = item.sender_name
            adjuntos = item.get_number_of_attachments
                        
            # Escribir los datos del correo en el archivo CSV
            csv_writer.writerow([asunto, fecha, enviado_por,adjuntos])
            
            count += 1  # Incrementar el contador de correos encontrados

        # Si hay carpetas anidadas, buscar recursivamente en ellas y sumar al contador
        if isinstance(item, pypff.folder):
            count += listar_correos(item, csv_writer)
    return count

# Ruta al archivo PST
archivo_pst = './data/backup.pst'

pst = pypff.file()
pst.open(archivo_pst)

# Acceder a la raíz del archivo PST
root_folder = pst.get_root_folder()

# Ruta al archivo CSV intermedio
archivo_csv = './data/backup.csv'

# Convertir PST a CSV
with open(archivo_csv, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['asunto', 'fecha', 'enviado_por','adjuntos'])
    total_correos = listar_correos(root_folder, csv_writer)

print(f'Se encontraron {total_correos} correos electrónicos y se guardaron en {archivo_csv}')


Se encontraron 6015 correos electrónicos y se guardaron en ./data/backup.csv


In [ ]:
import fileinput
import sys

# Ruta al archivo PST
archivo_pst = './data/backup.pst'

with open(archivo_pst, 'rb') as f:
    pst_data = f.read()

In [16]:
#Este script usa la primera version 
import pypff
import time
import pandas as pd
archivo_pst = './data/backup.pst'

df=pd.DataFrame(columns=['asunto', 'fecha', 'enviado_por'])

try:
    # Abrir el archivo PST
    start=time.time()
    pst = pypff.file()
    pst.open(archivo_pst)
    
    # Acceder a la raíz del archivo PST
    root_folder = pst.get_root_folder()

    # Solicitar palabra clave por consola
    palabra_clave = "unal"
    
    # Función para listar los correos electrónicos que contienen la palabra clave y contar su cantidad
    def listar_correos_con_palabra_clave(folder):
        count = 0  # Inicializar contador de correos encontrados en esta carpeta
        for item in folder.sub_items:
            # Verificar si es un correo electrónico
            if isinstance(item, pypff.message):
                if palabra_clave.lower() in item.subject.lower():
                    df.loc[count]={
                        'asunto': item.subject,
                        'fecha': item.delivery_time,
                        'enviado_por': item.sender_name
                    }
                    count += 1  # Incrementar el contador de correos encontrados
            
            # Si hay carpetas anidadas, buscar recursivamente en ellas y sumar al contador
            if isinstance(item, pypff.folder):
                count += listar_correos_con_palabra_clave(item)
        return count
    
    # Contar los correos electrónicos que contienen la palabra clave en el archivo PST
    total_correos = listar_correos_con_palabra_clave(root_folder)
    end=time.time()
    
    print(f"Se encontraron {total_correos} correos con la palabra clave en el asunto en el archivo PST.")
    print(f"Tiempo de ejecución: {end-start} segundos.")
    print(df)
    # Cerrar el archivo PST
    pst.close()
except Exception as e:
    print(f"Ocurrió un error: {str(e)}")

Se encontraron 802 correos con la palabra clave en el asunto en el archivo PST.
Tiempo de ejecución: 5.85771632194519 segundos.
                                                asunto               fecha  \
0    [ FEBRERO 22 ] Bienvenida UNAL admitidos 2020-... 2020-02-07 22:35:28   
1    ¡Cuéntanos cómo te gustaría ser denominado com... 2020-02-10 14:01:01   
2    ¿Cómo te gustaría ser denominado por ser miemb... 2020-02-10 14:02:38   
3    #ComunidadUNAL ¿Conoces la señalización de la ... 2020-02-10 14:02:40   
4    [ FEBRERO 22 ] Bienvenida UNAL admitidos 2020-... 2020-02-11 14:08:10   
..                                                 ...                 ...   
797  #SeñalizaciónUNAL ¡Juntos construimos la image... 2021-06-29 19:31:02   
798    Talentosa mezzosoprano UNAL | #SomosOrgulloUNAL 2021-06-30 14:32:15   
799  ¡Tú eliges el mejor momento para conectarte co... 2021-06-30 19:30:38   
800  Ciclo de conferencias UNAL y Argos te invita a... 2021-06-30 20:15:33   
801  Lleva a n

In [22]:
#Este script usa pyspark para realizar el filtrado
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

# Inicializar Spark
spark = SparkSession.builder.appName("FiltrarCorreos").getOrCreate()

# Ruta al archivo CSV
archivo_csv = './data/backup.csv'

start=time.time()
# Leer el archivo CSV como un DataFrame de Spark
df = spark.read.csv(archivo_csv, header=True, inferSchema=True)

# Palabra clave a buscar
palabra_clave = 'UnAL'

# Aplicar el filtro por la palabra clave en el asunto o el cuerpo
df_filtrado = df.filter(col('Asunto').contains(palabra_clave.lower()))

# Mostrar resultados
df_filtrado.show(df.count(), truncate=False)
end=time.time()
print(f"Se encontraron {df_filtrado.count()} correos con la palabra clave en el asunto o el cuerpo en el archivo CSV.")
print(f"Tiempo de ejecución: {end-start} segundos.")
spark.stop()

+-------------------------------------------------------------------------------------------------------------+-------------------+-----------------------------------------------+
|asunto                                                                                                       |fecha              |enviado_por                                    |
+-------------------------------------------------------------------------------------------------------------+-------------------+-----------------------------------------------+
|Re: Invitación: El Agua-Ecosistemas Acuáticos mié 22 de abr de 2020 9am - 11am (COT) (dposadal@unal.edu.co)  |2020-04-24 00:00:20|Diana Carolina Posada Lopez                    |
|Re: Invitación: Ecología de la conservación II mié 3 de jun de 2020 9am - 11am (COT) (dapalacios@unal.edu.co)|2020-06-03 14:04:29|Darwin Jair Palacios Plaza                     |
|💻 Ciberseguridad al alcance de todos con #i40unal.                                                 